In [6]:
import pandas as pd
import plotnine as gg 
import numpy as np
import scipy.stats as st
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, TargetEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import joblib
from sklearn.cluster import KMeans


from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, RepeatedStratifiedKFold

from preprocessing import *
sedd = 11

In [7]:
train = pd.read_csv("train.csv")
train = train.drop_duplicates(keep = 'first').copy()
train.reset_index(inplace = True)
y = train.Exited.copy()
X = train.drop('Exited', axis = 1).copy()
test = pd.read_csv("test.csv")

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   index            165034 non-null  int64  
 1   id               165034 non-null  int64  
 2   CustomerId       165034 non-null  int64  
 3   Surname          165034 non-null  object 
 4   CreditScore      165034 non-null  int64  
 5   Geography        165034 non-null  object 
 6   Gender           165034 non-null  object 
 7   Age              165034 non-null  float64
 8   Tenure           165034 non-null  int64  
 9   Balance          165034 non-null  float64
 10  NumOfProducts    165034 non-null  int64  
 11  HasCrCard        165034 non-null  float64
 12  IsActiveMember   165034 non-null  float64
 13  EstimatedSalary  165034 non-null  float64
 14  Exited           165034 non-null  int64  
dtypes: float64(5), int64(7), object(3)
memory usage: 18.9+ MB


In [9]:
train.head(3)

,index,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.0,2,1.0,0.0,181449.97,0
1,1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.0,2,1.0,1.0,49503.50,0
2,2,2,15694510,Hsueh,678,France,Male,40.0,10,0.0,2,1.0,0.0,184866.69,0


In [5]:
notes = {} 

In [62]:
# Age vs Geography
tt = train.copy()
one_k = KMeans(n_clusters = 4, n_init = 10)
one_geo = LabelEncoder() 
one_gender = LabelEncoder() 
one_scale = StandardScaler()
tt.Geography = one_geo.fit_transform(tt.Geography)
tt.Gender = one_gender.fit_transform(tt.Gender) 
tt.Age = np.log(tt.Age)
output = one_scale.fit_transform(tt[['Geography', 'Gender', 'Age']])

output2 = one_k.fit_predict(output)

In [63]:
output3 = pd.DataFrame({'cluster':output2, 'Exited': y})

In [64]:
pd.crosstab(output3.cluster, output3.Exited)

Exited,0,1
cluster,,
0,30665,8246
1,30974,9002
2,20805,11103
3,47669,6570


In [48]:
class InteractionTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        feat_1 = X.IsActiveMember * X.HasCrCard * tt.Tenure
        return np.array(feat_1).reshape(-1,1)

In [49]:
InteractionTransformer().fit_transform(X)

array([[0.],
       [1.],
       [0.],
       ...,
       [5.],
       [0.],
       [0.]])